# NER in CONLL format

In [28]:
label_names = ['O', 'B-MAT', 'I-MAT', 'B-NMAT', 'I-NMAT', 'B-DIMENSION', 'I-DIMENSION', 'B-COMPONENT', 'I-COMPONENT', 'B-MAIN_PRODUCT', 'I-MAIN_PRODUCT', 'B-WEIGHT', 'I-WEIGHT', 'B-TARGET_USER', 'I-TARGET_USER', 'B-PROPERTY', 'I-PROPERTY', 'B-COLOR', 'I-COLOR', 'B-SHAPE', 'I-SHAPE']
print(label_names)
print(len(label_names))

ner_tags_2_number = {t: i for (i, t) in enumerate(label_names)}
ner_tags_2_number

['O', 'B-MAT', 'I-MAT', 'B-NMAT', 'I-NMAT', 'B-DIMENSION', 'I-DIMENSION', 'B-COMPONENT', 'I-COMPONENT', 'B-MAIN_PRODUCT', 'I-MAIN_PRODUCT', 'B-WEIGHT', 'I-WEIGHT', 'B-TARGET_USER', 'I-TARGET_USER', 'B-PROPERTY', 'I-PROPERTY', 'B-COLOR', 'I-COLOR', 'B-SHAPE', 'I-SHAPE']
21


{'O': 0,
 'B-MAT': 1,
 'I-MAT': 2,
 'B-NMAT': 3,
 'I-NMAT': 4,
 'B-DIMENSION': 5,
 'I-DIMENSION': 6,
 'B-COMPONENT': 7,
 'I-COMPONENT': 8,
 'B-MAIN_PRODUCT': 9,
 'I-MAIN_PRODUCT': 10,
 'B-WEIGHT': 11,
 'I-WEIGHT': 12,
 'B-TARGET_USER': 13,
 'I-TARGET_USER': 14,
 'B-PROPERTY': 15,
 'I-PROPERTY': 16,
 'B-COLOR': 17,
 'I-COLOR': 18,
 'B-SHAPE': 19,
 'I-SHAPE': 20}

In [29]:
import pandas as pd

file_path_colnn = "../data/label_studio_2_v2_1044.conll"
file_colnn = open(file_path_colnn, 'r')
lines = file_colnn.readlines()

list_words = []
list_labels = []

word_pieces = []
label_pieces = []

for line in lines:
    line = line.strip()
    line_split = line.split(" ")

    if line_split[0] == "-DOCSTART-":
        continue
    elif line_split[0] == "":
        list_words.append(word_pieces)
        list_labels.append(label_pieces)

        word_pieces = []
        label_pieces = []
        continue

    word = line_split[0]
    label = ner_tags_2_number[line_split[3]]

    word_pieces.append(word)
    label_pieces.append(label)

raw_dataset = pd.DataFrame()
raw_dataset['tokens'] = pd.Series(list_words)
raw_dataset['ner_tags'] = pd.Series(list_labels)

In [30]:
from datasets import Dataset, DatasetDict, ClassLabel, Features, Value, Sequence

train_dataset = Dataset.from_pandas(raw_dataset)
val_dataset = Dataset.from_pandas(raw_dataset)
test_dataset = Dataset.from_pandas(raw_dataset)

raw_datasets = DatasetDict(
    {"train": train_dataset, "val": val_dataset, "test": test_dataset})
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 1044
    })
    val: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 1044
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 1044
    })
})

In [31]:
raw_datasets['train'][0]

{'tokens': ['🐶',
  '【',
  'support',
  'and',
  'piping',
  'design',
  '】',
  ':',
  'supportive',
  'raised',
  'sides',
  'of',
  'the',
  'rectangular',
  'pet',
  'bed',
  'create',
  'a',
  'safe',
  'space',
  'for',
  'your',
  'cats',
  'and',
  'dogs',
  'to',
  'lay',
  'their',
  'heads',
  ',',
  'allowing',
  'your',
  'pet',
  'to',
  'have',
  'a',
  'restful',
  'sleep',
  '.'],
 'ner_tags': [0,
  0,
  0,
  0,
  15,
  16,
  0,
  0,
  0,
  15,
  16,
  0,
  0,
  15,
  13,
  9,
  0,
  0,
  0,
  0,
  0,
  0,
  13,
  0,
  13,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  13,
  0,
  0,
  0,
  0,
  0,
  0]}

In [39]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [32]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

🐶 【 support and piping     design     】 : supportive raised     sides      of the rectangular pet           bed            create a safe space for your cats          and dogs          to lay their heads , allowing your pet           to have a restful sleep . 
O O O       O   B-PROPERTY I-PROPERTY O O O          B-PROPERTY I-PROPERTY O  O   B-PROPERTY  B-TARGET_USER B-MAIN_PRODUCT O      O O    O     O   O    B-TARGET_USER O   B-TARGET_USER O  O   O     O     O O        O    B-TARGET_USER O  O    O O       O     O 


In [33]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [34]:
tokenizer.is_fast

True

In [35]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 '[UNK]',
 '[UNK]',
 'support',
 'and',
 'p',
 '##ip',
 '##ing',
 'design',
 '[UNK]',
 ':',
 'supportive',
 'raised',
 'sides',
 'of',
 'the',
 'rectangular',
 'pet',
 'bed',
 'create',
 'a',
 'safe',
 'space',
 'for',
 'your',
 'cats',
 'and',
 'dogs',
 'to',
 'lay',
 'their',
 'heads',
 ',',
 'allowing',
 'your',
 'pet',
 'to',
 'have',
 'a',
 'rest',
 '##ful',
 'sleep',
 '.',
 '[SEP]']

In [36]:
inputs.word_ids()

[None,
 0,
 1,
 2,
 3,
 4,
 4,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 36,
 37,
 38,
 None]

In [37]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [38]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 0, 0, 15, 16, 0, 0, 0, 15, 16, 0, 0, 15, 13, 9, 0, 0, 0, 0, 0, 0, 13, 0, 13, 0, 0, 0, 0, 0, 0, 0, 13, 0, 0, 0, 0, 0, 0]
[-100, 0, 0, 0, 0, 15, 16, 16, 16, 0, 0, 0, 15, 16, 0, 0, 15, 13, 9, 0, 0, 0, 0, 0, 0, 13, 0, 13, 0, 0, 0, 0, 0, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0, -100]


In [39]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [40]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map: 100%|██████████| 1044/1044 [00:00<00:00, 18903.54 examples/s]


In [41]:
tokenized_datasets['train'][0]

{'input_ids': [101,
  100,
  100,
  1619,
  1105,
  185,
  9717,
  1158,
  1902,
  100,
  131,
  17847,
  2120,
  3091,
  1104,
  1103,
  10848,
  11109,
  1908,
  2561,
  170,
  2914,
  2000,
  1111,
  1240,
  11771,
  1105,
  6363,
  1106,
  3191,
  1147,
  4075,
  117,
  3525,
  1240,
  11109,
  1106,
  1138,
  170,
  1832,
  2365,
  2946,
  119,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [-100,
  0,
  0,
  0,
  0,
  15,
  16,
  16,
  16,
  0,
  0,
  0,
  15,
  16,
  0,
  0,
  15,
  13,
  9,
  0,
  0,
  0,
  0,
  0,
  0,
  13,
  0,
  13,

In [42]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [43]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    0,    0,    0,    0,   15,   16,   16,   16,    0,    0,    0,
           15,   16,    0,    0,   15,   13,    9,    0,    0,    0,    0,    0,
            0,   13,    0,   13,    0,    0,    0,    0,    0,    0,    0,   13,
            0,    0,    0,    0,    0,    0,    0, -100],
        [-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,   15,   16,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    1,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0, -100, -100, -100, -100, -100]])

In [44]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 0, 0, 0, 0, 15, 16, 16, 16, 0, 0, 0, 15, 16, 0, 0, 15, 13, 9, 0, 0, 0, 0, 0, 0, 13, 0, 13, 0, 0, 0, 0, 0, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0, -100]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [45]:
import evaluate

metric = evaluate.load("seqeval")

ModuleNotFoundError: No module named 'evaluate'

In [55]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [56]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'MISC': {'precision': 1.0,
  'recall': 0.5,
  'f1': 0.6666666666666666,
  'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 0.6666666666666666,
 'overall_f1': 0.8,
 'overall_accuracy': 0.8888888888888888}

In [57]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [58]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [59]:
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [60]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [61]:
model.config.num_labels

9

In [62]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [63]:
data_collator

DataCollatorForTokenClassification(tokenizer=BertTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True), padding=True, max_length=None, pad_to_multiple_of=None, label_pad_token_id=-100, return_tensors='pt')

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    'bert-finetuned-ner/checkpoint-5268',
    id2label=id2label,
    label2id=label2id,
)

2023-08-15 13:31:47.390083: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-15 13:31:47.423811: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-15 13:31:47.801330: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
from transformers import get_schedulerLong

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "bert-finetuned-ner-accelerate"
repo_name = get_full_repo_name(model_name)
repo_name

'sgugger/bert-finetuned-ner-accelerate'

In [ ]:
output_dir = "bert-finetuned-ner-accelerate"
repo = Repository(output_dir, clone_from=repo_name)

In [ ]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [ ]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

In [ ]:
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "bert-finetuned-ner/checkpoint-5268"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

Using /home/tanluuuuuuu/.cache/torch_extensions/py38_cu117 as PyTorch extensions root...
Creating extension directory /home/tanluuuuuuu/.cache/torch_extensions/py38_cu117/cuda_kernel...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/tanluuuuuuu/.cache/torch_extensions/py38_cu117/cuda_kernel/build.ninja...
Building extension module cuda_kernel...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/4] /usr/local/cuda-11.8/bin/nvcc  -DTORCH_EXTENSION_NAME=cuda_kernel -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/tanluuuuuuu/miniconda3/envs/one_for_all/lib/python3.8/site-packages/torch/include -isystem /home/tanluuuuuuu/miniconda3/envs/one_for_all/lib/python3.8/site-packages/torch/include/torch/csrc/api/include -isystem /home/tanluuuuuuu/miniconda3/envs/one_for_all/lib/python3.8/site-packages/torch/include/TH -isystem /home/tanluuuuuuu/miniconda3/envs/one_for_all/lib/python3.8/site-packages/torch/include/THC -isystem /usr/local/cuda-11.8/include -isystem /home/tanluuuuuuu/miniconda3/envs/one_for_all/include/python3.8 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS__ -D__CUDA_NO_BFLOAT16_CONVERSIONS__ -D__CUDA_NO_HALF2_OPERATORS__ --expt-relaxed-constexpr -gencode=arch=compute_86,code=compute_86 -gencode=arch=compute_86,code=sm_86 --compile

Loading extension module cuda_kernel...


[{'entity_group': 'PER',
  'score': 0.99878144,
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.9772945,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.99921894,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

In [ ]:
token_classifier("Sylvain, Hugging Face, Brooklyn.")

[{'entity_group': 'PER',
  'score': 0.7124092,
  'word': 'Sylva',
  'start': 0,
  'end': 5},
 {'entity_group': 'LOC',
  'score': 0.37630534,
  'word': '##in',
  'start': 5,
  'end': 7},
 {'entity_group': 'LOC',
  'score': 0.9712114,
  'word': 'Hugging Face',
  'start': 9,
  'end': 21},
 {'entity_group': 'LOC',
  'score': 0.99908817,
  'word': 'Brooklyn',
  'start': 23,
  'end': 31}]

In [ ]:
token_classifier("It's made from Long.")

[{'entity_group': 'MISC',
  'score': 0.5854402,
  'word': 'Long',
  'start': 15,
  'end': 19}]

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Apple shares rose on the news. Apple pie is delicious.")
doc.ents

(Apple, Apple)